In [3]:
pip install faker

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 1.8/1.8 MB 4.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
import pandas as pd
import numpy as np
from faker import Faker
import random

fake = Faker()

def generate_synthetic_data(num_records):
    data = []
    for _ in range(num_records):
        employee_id = fake.uuid4()
        task_date = fake.date_this_year()
        task_time = fake.time()
        task_type = random.choice(['customer_service', 'order_processing', 'inventory_management'])
        duration = random.randint(10, 180)  # Duration in minutes
        orders_processed = random.randint(0, 70)
        complaints_handled = random.randint(0, 10)
        feedback = random.choice(['positive', 'negative', 'neutral'])
        response_time = random.randint(1, 60)  # Response time in minutes
        
        data.append([employee_id, task_date, task_time, task_type, duration, orders_processed, complaints_handled, feedback, response_time])
    
    columns = ['EmployeeID', 'TaskDate', 'TaskTime', 'TaskType', 'Duration', 'OrdersProcessed', 'ComplaintsHandled', 'Feedback', 'ResponseTime']
    return pd.DataFrame(data, columns=columns)

# Generate 100,000 records
synthetic_data = generate_synthetic_data(200000)
synthetic_data.to_csv('synthetic_ecommerce_employee_performance_data.csv', index=False)


In [1]:
import pandas as pd
import random
from faker import Faker

fake = Faker()

# Carregar os dados sintéticos
synthetic_data = pd.read_csv('synthetic_ecommerce_employee_performance_data.csv')

# Criar colunas com nomes e departamentos falsos para a DimEmployee
def generate_employee_data(employee_ids):
    employees = []
    departments = ['Sales', 'Support']
    for emp_id in employee_ids:
        name = fake.name()
        department = random.choice(departments)
        employees.append([emp_id, name, department])
    return pd.DataFrame(employees, columns=['EmployeeID', 'EmployeeName', 'Department'])

# Obter IDs únicos dos funcionários
unique_employee_ids = synthetic_data['EmployeeID'].unique()

# Gerar dados de funcionários
dim_employee = generate_employee_data(unique_employee_ids)

# Tabela dimensão Tarefa
dim_task = synthetic_data[['TaskType']].drop_duplicates().reset_index(drop=True)
dim_task['TaskTypeID'] = dim_task.index + 1  # Adicionar ID único para cada tipo de tarefa

# Tabela dimensão Feedback
dim_feedback = synthetic_data[['Feedback']].drop_duplicates().reset_index(drop=True)
dim_feedback['FeedbackID'] = dim_feedback.index + 1  # Adicionar ID único para cada tipo de feedback

# Mesclar IDs únicos de tarefa e feedback na tabela fato
fact_table = synthetic_data.merge(dim_task, on='TaskType').merge(dim_feedback, on='Feedback')

# Selecionar colunas relevantes para a tabela fato
fact_table = fact_table[['EmployeeID', 'TaskDate', 'TaskTime', 'TaskTypeID', 'Duration', 'OrdersProcessed', 'ComplaintsHandled', 'FeedbackID', 'ResponseTime']]

# Salvar tabelas dimensão e fato em arquivos CSV
dim_employee.to_csv('dim_employee.csv', index=False)
dim_task.to_csv('dim_task.csv', index=False)
dim_feedback.to_csv('dim_feedback.csv', index=False)
fact_table.to_csv('fact_table.csv', index=False)
